In [61]:
# Example of how to scale any process over a large area
# GEE tends to run out of memory if complicated exports are run over a large area
# In order to avoid these errors, you may have to break up exports into smaller areas you manage instead of relying on GEE to figure it out for you
####################################################################################################

try:
  import geeViz.getImagesLib as getImagesLib
except:
  !python -m pip install geeViz
  import geeViz.getImagesLib as getImagesLib


#Module imports
# import geeViz.getImagesLib as getImagesLib
import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
ee = getImagesLib.ee

#Set up to mapper objects to use
#Can use the default one first
Map = getImagesLib.Map

# Enter proxy url (see in url bar- e.g. https://someCode-dot-someRegion.notebooks.googleusercontent.com/)
# If left as None, will prompt user at first Map.view() call 
Map.proxy_url = 'https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/'
print('done')

done


In [62]:
Map.clearMap()

# First, we'll view the tiles used in the current CONUS LCMS workflow
lcms_CONUS_composites = ee.ImageCollection('projects/lcms-tcc-shared/assets/Composites/Composite-Collection-yesL7-1984-2020')\
                                                .filter(ee.Filter.calendarRange(2022,2022,'year'))

# Pull the geometry of each tile in the composites
lcms_composites_tile_geo = lcms_CONUS_composites.map(lambda f:ee.Feature(f.geometry()).copyProperties(f,['studyAreaName']))

# Add the tiles and a composite for reference
Map.addLayer(lcms_CONUS_composites.mosaic(),getImagesLib.vizParamsTrue10k,'Example CONUS 2022 LCMS Composite')
Map.addLayer(lcms_composites_tile_geo,{},'LCMS Composite Tile Geometry')

Map.centerObject(lcms_composites_tile_geo)
Map.turnOnInspector()
Map.view()

Adding layer: Example CONUS 2022 LCMS Composite
Adding layer: LCMS Composite Tile Geometry
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [63]:
Map.clearMap()

# Here is an example of how to create a pyramid of tiles at various scales
# Generally you would start with the biggest tile possible and work your way down till you stop having memory issues
# Currently, LCMS uses 480km tiles (with a 900m buffer) for most processing
lcms_CONUS_studyArea = ee.FeatureCollection('projects/lcms-292214/assets/CONUS-Ancillary-Data/conus')
lcms_CONUS_projection = lcms_CONUS_composites.first().projection()

def getGrid(studyArea,projection,size):
  grid = studyArea.geometry().coveringGrid(projection.atScale(size))
  Map.addLayer(grid,{},'Tile Grid {}m'.format(size))
  return grid

grid480= getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,480000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,240000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,120000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,60000)
Map.addLayer(lcms_CONUS_studyArea,{},'LCMS CONUS Study Area')

Map.turnOnInspector()
Map.view()


Adding layer: Tile Grid 480000m
Adding layer: Tile Grid 240000m
Adding layer: Tile Grid 120000m
Adding layer: Tile Grid 60000m
Adding layer: LCMS CONUS Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [64]:
Map.clearMap()
# As an example of how to use this approach, we first need to get each available tile into a list
# Then we iterate across each tile and get the data and export, etc...
ids = grid480.limit(2).aggregate_histogram('system:index').keys().getInfo()
for id in ids:
  print('ID:',id)
  tile = grid480.filter(ee.Filter.eq('system:index',id)).geometry().buffer(900)
  Map.addLayer(tile,{},'Tile {}'.format(id))

Map.view()

ID: -3,-6
Adding layer: Tile -3,-6
ID: -4,-6
Adding layer: Tile -4,-6
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [65]:
# Since CCDC is the most memory intensive algorith we use, you are the most likely to need to use this approach when running CCDC
# The previous CCDC example ran from 2010-2023 and did not run out of memory
# If you run it from 1984 to 2023 using just Landsat, you are much more likely to run out of memory

Map.clearMap()

# First let's set up some Landsat and CCDC parameters

#Update the startJulian and endJulian variables to indicate your seasonal
#constraints. This supports wrapping for tropics and southern hemisphere.
#If using wrapping and the majority of the days occur in the second year, the system:time_start will default
#to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
#startJulian: Starting Julian date
#endJulian: Ending Julian date
startJulian = 1
endJulian = 365

#Specify start and end years for all analyses
#More than a 3 year span should be provided for time series methods to work
#well. If providing pre-computed stats for cloudScore and TDOM, this does not
#matter
startYear = 1984
endYear = 2023


#Choose whether to include Landat 7
#Generally only included when data are limited
includeSLCOffL7 = True


#Which bands/indices to export
#These will not always be used to find breaks - that is specified below in the ccdcParams
#Options are: ["blue","green","red","nir","swir1","swir2","NDVI","NBR","NDMI","NDSI","brightness","greenness","wetness","fourth","fifth","sixth","tcAngleBG"]
#Be sure that any bands in ccdcParams.breakpointBands are in this list
exportBands = ["blue","green","red","nir","swir1","swir2","NDVI"]

###############################################################
#CCDC Params
# ccdcParams ={
#   breakpointBands:['green','red','nir','swir1','swir2','NDVI'],//The name or index of the bands to use for change detection. If unspecified, all bands are used.//Can include: 'blue','green','red','nir','swir1','swir2'
#                                                               //'NBR','NDVI','wetness','greenness','brightness','tcAngleBG'
#   tmaskBands : None,//['green','swir2'],//The name or index of the bands to use for iterative TMask cloud detection. These are typically the green band and the SWIR2 band. If unspecified, TMask is not used. If specified, 'tmaskBands' must be included in 'breakpointBands'.,
#   minObservations: 6,//Factors of minimum number of years to apply new fitting.
#   chiSquareProbability: 0.99,//The chi-square probability threshold for change detection in the range of [0, 1],
#   minNumOfYearsScaler: 1.33,//Factors of minimum number of years to apply new fitting.,\
#   lambda: 0.002,//Lambda for LASSO regression fitting. If set to 0, regular OLS is used instead of LASSO
#   maxIterations : 25000, //Maximum number of runs for LASSO regression convergence. If set to 0, regular OLS is used instead of LASSO.
#   dateFormat : 1 //'fractional' (1) is the easiest to work with. It is the time representation to use during fitting: 0 = jDays, 1 = fractional years, 2 = unix time in milliseconds. The start, end and break times for each temporal segment will be encoded this way.

# };
ccdcParams ={
  'breakpointBands':['green','red','nir','swir1','swir2','NDVI'],
  'tmaskBands' : None,
  'minObservations': 6,
  'chiSquareProbability': 0.99,
  'minNumOfYearsScaler': 1.33,
  'lambda': 0.002,
  'maxIterations' : 25000,
  'dateFormat' : 1
};


# Output collection
tiledCCDCOutputCollection = 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4'

# Make sure collection exists
aml.create_image_collection(tiledCCDCOutputCollection)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(tiledCCDCOutputCollection,writers = [],all_users_can_read = True,readers = [])

# We'll use the full Puerto Rico and US Virgin Islands LCMS study area
studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

# Set the size (in meters) of the tiles
tileSize = 60000

# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))


#Remove any extremely high band/index values
def removeGT1(img):
  lte1 = img.select(['blue','green','nir','swir1','swir2']).lte(1).reduce(ee.Reducer.min());
  return img.updateMask(lte1);

getImagesLib.vizParamsFalse['min']=0.15
getImagesLib.vizParamsFalse['max']=0.8


Map.addLayer(studyArea,{},'Study Area')
Map.turnOnInspector()
Map.view()

Collection projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4 already exists
Updating permissions for:  projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4
Adding layer: Tile Grid 60000m
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [66]:
# Now, we'll iterate across each tile and run CCDC


# As an example of how to use this approach to create composites
ids = grid.aggregate_histogram('system:index').keys().getInfo()
for id in ids:
    print(id)
    # Get the tile and buffer it so there are no missing pixels at tile edges
    tile = grid.filter(ee.Filter.eq('system:index',id)).geometry().intersection(studyArea,240,projection).buffer(900)
    
    
    # Map.addLayer(tile,{},'Tile {}'.format(id))
    
    processedScenes = getImagesLib.getProcessedLandsatScenes(studyArea = tile,startYear = startYear, endYear = endYear,
                                                        startJulian = startJulian,endJulian = endJulian,
                                                        includeSLCOffL7 = includeSLCOffL7).select(exportBands)
    processedScenes = processedScenes.map(removeGT1)
    # print(processedScenes.size().getInfo())
    
    #Set the scene collection in the ccdcParams
    ccdcParams['collection'] = processedScenes

    #Run CCDC
    ccdc = ee.Image(ee.Algorithms.TemporalSegmentation.Ccdc(**ccdcParams))
    ccdc = ccdc.set({'startYear':startYear,
                     'endYear':endYear,
                     'startJulian':startJulian,
                     'endJulian':endJulian,
                     'TileSize':tileSize,
                     'TileID':id})
    
    
    exportName = 'CCDC_Tile-{}m_ID{}_yrs{}-{}_jds{}-{}'.format(tileSize,id.replace(',','-'),startYear,endYear,startJulian,endJulian)
    exportPath = tiledCCDCOutputCollection + '/'+ exportName
    print(exportPath)

    getImagesLib.exportToAssetWrapper(ccdc,exportName,exportPath,{'.default':'sample'},tile,scale,crs,transform,overwrite=False)



90,54
Get Processed Landsat: 
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID90-54_yrs1984-2023_jds1-365
pyramiding object: {'.default': 'sample'}
Asset currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
91,52
Get Processed Landsat: 
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_

KeyboardInterrupt: 

In [67]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(exportPathRoot, type = 'imageCollection')

print('done')

done


In [68]:
Map.clearMap()

# Bring in the outputs and mosaic them into a single image
ccdcImg = ee.ImageCollection(tiledCCDCOutputCollection).mosaic()
Map.addLayer(ccdcImg,{},'CCDC Raw Image')

Map.turnOnInspector()
Map.view()

Adding layer: CCDC Raw Image
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [73]:
# Same workflow follows as with an untiled CCDC output
Map.clearMap()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False

#Apply the CCDC harmonic model across a time series
#First get a time series of time images with a time step of 0.1 of a year
yearImages = changeDetectionLib.getTimeImageCollection(startYear,endYear,startJulian,endJulian,0.1);

#Then predict the CCDC models
fitted = changeDetectionLib.predictCCDC(ccdcImg,yearImages,fillGaps,whichHarmonics)
Map.addLayer(fitted.select(['.*_predicted']),{'bands':'swir1_predicted,nir_predicted,red_predicted','min':0.05,'max':0.6},'Fitted CCDC',True);


# Synthetic composites visualizing
# Take common false color composite bands and visualize them for the next to the last year

# First get the bands of predicted bands and then split off the name
fittedBns = fitted.select(['.*_predicted']).first().bandNames()
bns = fittedBns.map(lambda bn: ee.String(bn).split('_').get(0))

# Filter down to the next to the last year and a summer date range
syntheticComposites = fitted.select(fittedBns,bns)\
    .filter(ee.Filter.calendarRange(endYear-1,endYear-1,'year'))\
    .filter(ee.Filter.calendarRange(60,80)).first()

# Visualize output as you would a composite
Map.addLayer(syntheticComposites,getImagesLib.vizParamsFalse,'Synthetic Composite')

Map.turnOnInspector()
Map.view()


Adding layer: Fitted CCDC
Adding layer: Synthetic Composite
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None
